In [1]:
'''
    Esse código baixa dados do TD
    e grava em um planilha (privada) do google docs
'''

import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/spreadsheets']
jfile = 'carteira-328314-d38dcc8ee3e4.json'

credentials = ServiceAccountCredentials.from_json_keyfile_name(jfile, scope)
gc = gspread.authorize(credentials)

from datetime import datetime
import pandas as pd
titulo_planilha = 'Investimentos'
planilha = gc.open(titulo_planilha)
atualiza_saldos = True
pagina = planilha.worksheet('Renda Fixa')

In [2]:

from urllib.request import urlopen

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import json
url = 'https://www.tesourodireto.com.br/json/br/com/b3/tesourodireto/service/api/treasurybondsinfo.json'
response = urlopen(url)
data_json = json.loads(response.read())

HTTPError: HTTP Error 403: Forbidden

In [ ]:
nome , pr , pc , tr , tc = [],[],[],[],[]
td=pd.DataFrame()
for i in range(len(data_json["response"]["TrsrBdTradgList"])):
  nome.append(data_json["response"]["TrsrBdTradgList"][i]['TrsrBd']['nm'])
  pr.append(data_json["response"]["TrsrBdTradgList"][i]['TrsrBd']['untrRedVal'])
  pc.append(data_json["response"]["TrsrBdTradgList"][i]['TrsrBd']['untrInvstmtVal'])
  tr.append(data_json["response"]["TrsrBdTradgList"][i]['TrsrBd']['anulRedRate'])
  tc.append(data_json["response"]["TrsrBdTradgList"][i]['TrsrBd']['anulInvstmtRate'])

td['título'] = nome
td['preço resgate'] = pr
td['preço compra'] = pc
td['taxa resgate'] = tr
td['taxa compra'] = tc

td['Ano'] = td['título'].str.extract(r'\b(\d{4})\b')
td = td.sort_values(by=['título','Ano'], ascending=True)
# td.drop(['Ano'],axis=1,inplace=True)

td

In [ ]:
td.drop(['Ano'],axis=1,inplace=True)

In [ ]:
from datetime import date
today = date.today().strftime('%d/%m/%Y')

In [ ]:
linha = int(input('linha da celula inicial (79)'))
pagina.update_cell(linha, 1,  today )

In [ ]:
pagina.update('a'+str(linha+1),[td.columns.values.tolist()] + td.values.tolist())

In [ ]:
meus_tits = []
for linha in range(3,11):
    meus_tits.append(pagina.cell(linha, 1).value)
meus_tits

In [ ]:
meus = td[td['título'].isin(meus_tits)].copy()
meus['título']= meus['título'].str.replace('Tesouro ', '')
meus['título']= meus['título'].str.replace('com Juros Semestrais', 'Juros')
meus['título']= meus['título'].str.replace('20', '')

meus = meus.rename(columns={'título':str(today)})

meus

In [ ]:
pagina.update_cell(2, 13,  today )

In [ ]:
pagina.update('m2',[meus.columns.values.tolist()] + meus.values.tolist())